## Week 2 Day 3 - LAB contribution



### Added functionality

#### 1. Lead Qualification System with Structured Output
Add a lead scoring agent that evaluates prospects before sending emails:

- Create a ```LeadScore``` Pydantic model with fields  ```score``` (0-100), ```decision``` (send/skip), ```reasoning```, and ```suggested_tone```(professional/witty/concise)
- Build a lead qualification agent that takes company info (size, industry, funding stage) and outputs this structured format
- Instruct the sales manager to verify the lead score is above a threshold before proceeding
- The lead scorer chooses which sales_agent tool to use based on the prospect profile, instead of generating all 3 e-mails.

#### 2. Compliance & Brand Safety Layer
Extend the guardrail system for enterprise use:

- Create a ```ComplianceCheck``` structured output ```(violations: List[str], risk_level: str, approved: bool)```
- Build a compliance agent that checks for unsubstantiated claims, regulatory issues, or brand guideline violations
- Add both input guardrails (blocking certain industry targets) and output guardrails (verifying email meets compliance standards before sending)
- Create a "revision loop" where non-compliant emails get sent back to the sales agents with specific feedback for regeneration

#### 3. Added some context
1. Added context created during the previous lab exercise to help agents write better e-mails and make better decisions. This was necessary to avoid constant loops, as the compliance checker tends to be on the strict side without context (everything was unsubstantiated)
2. Tweaked the prompts

##### Caveats:
- Limited testing, there will be bugs (you have been warned!)

#### Updated flow: 

Lead Scoring → Single Email Generation → Compliance Check → (Guardrail) → Send

In [ ]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput, output_guardrail
from typing import Dict, List, Literal
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel, ConfigDict
from datetime import datetime, date
import json


In [ ]:
load_dotenv(override=True)

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [ ]:
# Create sales agent instructions dict

today = str(date.today())

with open('ComplAI_brochure.md', 'r') as file:
    brochure_text = file.read()

instructions_with_context_1 = f"""## Persona
You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 

## Context
Here is some information about the product which you can use: {brochure_text}

## Tone 
You write professional, serious cold emails. \nToday's date is {today}"""

instructions_with_context_2 = f"""## Persona
You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 

## Context
Here is some information about the product which you can use: {brochure_text}

## Tone 
You write witty, serious cold emails. \nToday's date is {today}"""

instructions_with_context_3 = f"""## Persona
You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 

## Context
Here is some information about the product which you can use: {brochure_text}

## Tone 
You write concise, serious cold emails. \nToday's date is {today}"""

brand_guidelines = f"""
\n## BRAND GUIDELINES:
- Never make unsubstantiated ROI claims
- Don't use superlatives like "best" or "revolutionary" 
- Focus on time-saving and audit preparation benefits
- Keep technical jargon minimal
- Always mention our SOC2 certification

## Personalization
Use these pain points to personalize: {{pain_points}}
"""

instructions_with_context_1 += brand_guidelines
instructions_with_context_2 += brand_guidelines
instructions_with_context_3 += brand_guidelines

AGENT_INSTRUCTIONS = {
    "professional": instructions_with_context_1,
    "witty": instructions_with_context_2,
    "concise": instructions_with_context_3
}

# Create generic sales e-mail generation tool
# Instead of generating 3 different e-mails we will generate the one that the lead_scoring tool will recommend

@function_tool
async def generate_sales_email(
    tone: str,
    pain_points: str,
    target_info: str = "") -> str: 

    """ Generate a personalized cold sales e-mail with the requested tone. 
    The e-mail covers the pain_points of the target described in target_info.
    """
    
    if(tone not in AGENT_INSTRUCTIONS):
        tone = 'concise' # Default to concise if key is not recognized
    
    instructions = AGENT_INSTRUCTIONS[tone].format(pain_points = pain_points)
    agent = Agent(
        name=f"{tone} Sales Agent",
        instructions=instructions,
        model="gpt-4o-mini"
    )
    
    result = await Runner.run(agent, f"Write a cold email addressing: {target_info}")
    return result.final_output

### It's easy to use any models with OpenAI compatible endpoints

In [ ]:
#GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
#DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
#GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [ ]:

#deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
#gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
#groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

#deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
#gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
#llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [ ]:
# Lead Scoring Agent

class LeadScore(BaseModel):
    score: int  # 0-100
    decision: str  # "send" or "skip"
    reasoning: str
    recommended_agent: str  # "sales_agent1", "sales_agent2", or "sales_agent3"
    key_pain_points: List[str]

lead_scorer_instructions = f"""You are scoring leads for a cold e-mail outreach campaign for ComplAI, 
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.
Analyze prospect info and determine:
    - Lead quality score (0-100)
    - Which sales agent tone fits best (professional/witty/concise)
    - Key pain points to address
    Only recommend sending if score >= 60
    
    Information about the tool that may help your decision: {brochure_text}"""

lead_scorer_agent = Agent(
    name="Lead Qualifier",
    instructions=lead_scorer_instructions,
    output_type=LeadScore,
    model="gpt-4o-mini"
)

lead_scorer = lead_scorer_agent.as_tool(tool_name = "lead_scorer", tool_description= "Perform Lead Scoring")

In [ ]:
# Compliance Checker 

class ComplianceCheck(BaseModel):
    model_config = ConfigDict(strict=True)
    
    approved: bool
    violations: List[str] = []  # empty if approved
    risk_level: Literal['low', 'medium', 'high'] # "low", "medium", "high"


compliance_checker_instructions = f"""Review email against brand guidelines:
    - Flag unsubstantiated claims (ROI %, time savings without data)
    - Check for superlatives/hyperbole (e.g. "guaranteed compliance", "absolute best", "no.1 in the market")
    - Marketing language is ok as long as 
    - Verify pain points are addressed professionally
    - Ensure ComplAI SOC2 certification is mentioned. This doesn't need further substantiation.
    - Assess email risk as 'low', 'medium' or 'high'
    Approve only if risk_level is 'low'    
    Respond with:
    - approved: (True/False)
    - violations: comma separated list of brand guidelines violations
    - risk_level: (low/medium/high)

    Product information:
    {brochure_text}
""" 

compliance_agent = Agent(
    name="compliance_checker",
    instructions=compliance_checker_instructions,
    output_type=ComplianceCheck,
    model="gpt-5-mini"
)

compliance_checker = compliance_agent.as_tool(tool_name = "compliance_checker", tool_description = "Check for brand compliance")

In [ ]:
#sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
#professional_sales_agent =  Agent(name="Professional Sales Agent", instructions=instructions_with_context_1, model="gpt-4o-mini")
#witty_sales_agent =  Agent(name="Witty Sales Agent", instructions=instructions_with_context_2, model="gpt-4o-mini")
#concise_sales_agent =  Agent(name="Concise Sales Agent", instructions=instructions_with_context_3, model="gpt-4o-mini")
#sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

In [ ]:
#description = "Write a cold sales email"


#tool1 = professional_sales_agent.as_tool(tool_name="professional_sales_agent", tool_description=description)
#tool2 = witty_sales_agent.as_tool(tool_name="witty_sales_agent", tool_description=description)
#tool3 = concise_sales_agent.as_tool(tool_name="concise_sales_agent", tool_description=description)

In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("lchanio@echyperion.com")  # Change to your verified sender
    to_email = To("proklos+sg@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
@function_tool
def add_email_signature(email_body: str, name: str = 'Sales Representative', phone :str = '', email:str = '') -> str:
    """ Add a professional email signature to the email body before HTML formatting """

    
    signature = f"\n\nBest regards,\n{name}"
    
    if phone:
        signature += f"\nPhone: {phone}"
    if email:
        signature += f"\nEmail: {email}"
    
    signature += "\n\nComplAI - AI-Powered SOC2 Compliance Solutions"
    signature += "\nMaking compliance simple, secure, and smart."
    
    return email_body + signature

In [ ]:
signature_instructions = "You add professional email signatures to email bodies. \
You are given an email body and signature information (name, phone, email) and you need to add a well-formatted signature."

signature_writer = Agent(name="Email signature writer", instructions=signature_instructions, model="gpt-4o-mini")
signature_tool = signature_writer.as_tool(tool_name="signature_writer", tool_description="Add a professional email signature to the email body")

In [ ]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [ ]:
email_tools = [subject_tool, add_email_signature, html_tool, send_html_email]

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
Follow these steps in order: \
1. Use the subject_writer tool to write a subject for the email \
2. Use the add_email_signature tool to add a professional signature (use default signature info: name='Alice Johnson', phone='+1-555-0123', email='alice@complai.com') \
3. Use the html_converter tool to convert the updated body with signature to HTML \
4. Finally, use the send_html_email tool to send the final email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-5-mini",
    handoff_description="Convert an email to HTML and send it")

In [ ]:
tools = [lead_scorer, generate_sales_email, compliance_checker]
handoffs = [emailer_agent]

In [ ]:
# Add a guardrail for always escalating high-risk e-mails. 

@output_guardrail  
async def prevent_high_risk_email_sending(ctx, agent, result):
    # Look for compliance check results in the conversation
    # Check if there's a high risk email that's being handed off

    # Type check - only proceed if there's a handoff
    if not hasattr(result, 'handoff') or not result.handoff:
        return GuardrailFunctionOutput(tripwire_triggered=False, output_info={})
    
    if result.handoff.name == "Email Manager":
        # Look for stored compliance results
        compliance_results = []
        for message in ctx.messages:
            if "COMPLIANCE_RESULT:" in str(message.content):
                compliance_json = str(message.content).split("COMPLIANCE_RESULT:")[1].split('\n')[0].strip()
                compliance_result = json.loads(compliance_json.lower())
                compliance_results.append(compliance_result)
        
        #use the most recent compliance result
        if (compliance_result[-1]['risk_level']=='high'):
            return GuardrailFunctionOutput(
                tripwire_triggered=True,
                output_info={"reason": "High risk email requires human review"}
            )
    
    return GuardrailFunctionOutput(tripwire_triggered=False, output_info={})

In [ ]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Get lead score using the lead_scorer tool passing the information about the lead.

2. If decision is "skip", stop and explain why

3. If decision is "send", use the generate_sales_email tool with:
- tone: the recommended_agent from lead scoring
- pain_points: the key_pain_points as a comma-separated string
- target_info: any relevant prospect details

4. Pass the single draft to compliance_checker

5. If approved, hand off to Email Manager

6. If not approved and risk is medium, regenerate the sales email once with violation feedback.

7. If high risk or second failure, stop and escalate to human review
 
Crucial Rules:
- You must use the generate_sales_email tool to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
- Never send the e-mail back for regeneration more than once. If there's a second compliance_checker failure, always stop and escalate to human review.
- After compliance check, store the result in your context in JSON format as 'COMPLIANCE_RESULT: {result}' before making handoff decision
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    output_guardrails = [prevent_high_risk_email_sending],
    handoffs=handoffs,
    model="gpt-5-mini")

message = "Send out a cold sales email addressed to John Doe, Chief Compliance Officer of SmartBank, a growing digital neo-bank in Belgium catering to young entrepreneurs in Europe, from Alice Johnson, Sales Representative at ComplAI."
message2 = "Send out a cold sales email addressed to Helen Writer, award-winning children's books author, from Alice Johnson, Sales Representative at ComplAI."

with trace("Automated SDR with Lead Scoring and Compliance"):
    result = await Runner.run(sales_manager, message2, max_turns=15) # The workflow may require more than the default 10 steps if there are multiple retries.

## Check out the trace:

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>